In [14]:
import random
from collections import Counter, defaultdict
from pymongo import MongoClient
import pickle
import os
import json
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from utils import *
from sklearn.metrics.pairwise import cosine_similarity
import jieba
import re
import matplotlib.pyplot as plt
from datetime import timedelta

In [1]:
class UserFeedback:
    def __init__(self, df_user):
        self.user_movie_feedback = {}
        for i, row in df_user.iterrows():
            key = (row['user_id'], row['movie_id'])
            if key in self.user_movie_feedback:
                print ("Error", key)
            self.user_movie_feedback[key] = dict(row)
    
    def get_ui_value(self, user_id, item_id, key="post_expected_preference"):
        ui_key = (user_id, item_id)
        if ui_key in self.user_movie_feedback:
            if key in self.user_movie_feedback[ui_key]:
                return int(self.user_movie_feedback[ui_key][key])
        # print ("Not found")
        return np.nan

class PeerAssessment:
    def __init__(self, df_peer):
        self.peer_user_movie_assessment = {}
        self.user_movie_peer_assessment = {}
        for i, row in df_peer.iterrows():
            key = (row['peer_id'], row['user_id'], row['movie_id'])
            if key in self.peer_user_movie_assessment:
                print ("Error", key)
            self.peer_user_movie_assessment[key] = dict(row)
            
            key = (row['user_id'], row['movie_id'])
            self.user_movie_peer_assessment.setdefault(key, {})
            self.user_movie_peer_assessment[key][row['peer_id']] = dict(row)
    
    def get_pui_value(self, peer_id, user_id, item_id, key="peer_preference"):
        pui_key = (peer_id, user_id, item_id)
        if pui_key in self.peer_user_movie_assessment:
            if key in self.peer_user_movie_assessment[pui_key]:
                return self.peer_user_movie_assessment[pui_key][key]
        # print ("Not found")
        return np.nan
    
    def get_ui_value(self, user_id, item_id, key="peer_preference", fun=lambda t: t[0]):
        ui_key = (user_id, item_id)
        if ui_key in self.user_movie_peer_assessment:
            ans = {}
            for p, vs in self.user_movie_peer_assessment[ui_key].items():
                ans[p] = vs[key]
            return ans
        return None

In [16]:
# if __name__ == "__main__":
#     user_feedback = UserFesedback(pd.read_pickle("data/user-study/bug/df_user.pkl"))
#     peer_assess = PeerAssessment(pd.read_pickle("data/user-study/bug/df_peer.pkl"))